**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 4 - Open Science Platform - Cross link User and Project accounts

1 - Load required modules

In [7]:
from new_helper import *
from super_helper import *
from loguru import logger

2 - Set variables and values

In [8]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [9]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 10:21:17,232 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 10:21:17.234 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [10]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-02-09 10:21:17.294 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:21:17.315 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 10:21:17.316 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 10:21:17.318 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:21:17.320 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-09 10:21:17,335 - INFO - None
2025-02-09 10:21:17,338 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-02-09 10:21:18,015 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:21:18,023 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:21:18.024 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 10:21:18.046 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 10:21:18.047 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-09 10:21:18.049 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-09 10:21:18.050 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-09 10:21:18.051 | DEBUG    | integration_helpers

7 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [11]:
#Query - GetAccountDetail

from iroha_helper import get_account_detail

user_details = get_account_detail(f"{user_account['account_id']}")

# logger.info(f'User Account id = {user_account}, {user_details}')

# # Process the account details response in JSON format
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object

# # Exposes the project_id 
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(f"Project Id = {project_id}")

2025-02-09 10:21:21.105 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account_detail"
2025-02-09 10:21:21,126 - INFO - User Account id = distracted_rhodes@test, { "admin@test" : { "account_metadata_cid" : "QmXBxKDZmmoKLjHkMHCK3rmezZSD9rGzp5EMfxQnK8Vmia", "linked_project" : "23897@test" } }
2025-02-09 10:21:21.128 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account_detail"
2025-02-09 10:21:21.129 | INFO     | __main__:<module>:14 - Project Id = 23897@test


In [13]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)

2025-02-09 10:37:43,965 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 10:37:43.968 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 10:37:43.971 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:37:43.999 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 10:37:44.001 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:37:44.003 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"


In [16]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, "distracted_rhodes", "test")
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-09 10:42:54.075 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:42:54,078 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:42:54,098 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:42:54,100 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:42:55,454 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:42:55,464 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:42:55,466 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'd1c3d0961d635bf7af313ec458006a3933221700edcf21f2a34091c6f9b5280d', 'timestamp': 1739097775466}
2025-02-09 10:42:55.468 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:42:55,471 - INFO - Appended data under 'distracted_rhodes@test' in logs/account_data.json
2025-02-09 10:42:55.472 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 

In [17]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, "23897", "test")
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details) #TO DO - From now

2025-02-09 10:43:19.663 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:43:19,668 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:43:19,685 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:43:19,687 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:43:22,511 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:43:22,520 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:43:22,523 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'f7517325370d12f0c29b2c5a0bf507a9d16a3c17902b8e7f741a7d8343c56610', 'timestamp': 1739097802523}
2025-02-09 10:43:22.524 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:43:22,527 - INFO - Appended data under '23897@test' in logs/account_data.json
2025-02-09 10:43:22.528 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 10:43:22.530

8 - Saves `project_id` in the local file sytem for use in Notebook 4

In [12]:
# Dumping the variable to JSON
dump_variable(project_id, "Project Id")

2025-02-09 10:21:21,147 - INFO - Variable 'Project Id' successfully dumped to temp/Project Id.json
